In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

CCAL version 0.9.1 @ /home/kwatme/github/ccal/ccal/__init__.py


In [3]:
if SETTING["signal_type"] != "raw":

    raise

In [4]:
feature_x_sample_processed = pd.read_table(
    PATH["feature_x_sample.processed.tsv"], index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [5]:
title_template = "{} Ranking by Raw Signal"

value_name = "Raw Signal Standard Deviation"

In [6]:
if SETTING["select_feature_automatically"]:

    features = ccal.select_series_indices(
        feature_x_sample_processed.std(axis=1),
        ">",
        standard_deviation=SETTING["select_feature_standard_deviation"],
        title=dict(text=title_template.format(feature_x_sample_processed.index.name)),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name),
    )

else:

    features = feature_x_sample_processed.index

features.size

77

In [7]:
if SETTING["select_sample_automatically"]:

    samples = ccal.select_series_indices(
        feature_x_sample_processed.std(),
        ">",
        standard_deviation=SETTING["select_sample_standard_deviation"],
        title=dict(text=title_template.format(feature_x_sample_processed.index.name)),
        xaxis=dict(title="Rank"),
        yaxis=dict(title=value_name),
    )

else:

    samples = feature_x_sample_processed.columns

samples.size

1074

In [8]:
raw_signal = feature_x_sample_processed.loc[features, samples]

In [9]:
if SETTING["signal_normalization_method"] is not None:

    raw_signal = ccal.normalize_df(
        raw_signal,
        SETTING["signal_normalization_axis"],
        SETTING["signal_normalization_method"],
        raise_for_bad=False,
    )

In [10]:
raw_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias=SETTING["feature_x_sample_alias"],
    feature_x_sample_value_name="Signal",
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_distributions_max_size=SETTING["plot_distributions_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)

Shape: (77, 1074)
Min: 0.0
Max: 13000.0
Axis 0 Number of Unique Values:
          Count
N Unique       
77          714
76          294
75           60
74            4
73            2
Axis 1 Top and Bottom 8 Number of Unique Values:
          Count
N Unique       
978           4
1001          4
1019          3
958           3
998           2
961           2
959           2
955           2
985           1
987           1
993           1
997           1
1004          1
1006          1
1011          1
1024          1


Sampling random 24,000 values ...


N NA: 0 (0.00%)
